In [1]:
import sqlite3
import pandas as pd

In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

In [ ]:
# Основная идея - в датасете rec-libimseti-dir.edges находим пользователей, с которыми рейтинг больше или равен 6, и 
# сравниваем этих пользователей с остальными из датасета info через косинусное сходство 

In [4]:
# выводим датасет rec-libimseti-dir.edges
df_ratings = pd.read_csv("E:/git_exercise/Final/edges/rec-libimseti-dir.edges", delim_whitespace=True, skiprows=1, names=["id_from", "id_to", "rating"])
df_ratings.head()

,id_from,id_to,rating
0,1,133,8
1,1,720,6
2,1,971,10
3,1,1095,7
4,1,1616,10


In [5]:
# создаем таблицу
df_ratings.to_sql('ratings', con, index=False, if_exists='replace')

17359346

In [6]:
#Выбор пользоватеелей, которых лайкнули пользователи с пятью заданными id (98447, 111944, 111930, 82091, 125299)
ratings_request=pd.read_sql(
    '''
    SELECT * FROM ratings
    WHERE ((id_from=98447) OR (id_from=111944) OR (id_from=111930) OR (id_from=82091) OR (id_from=125299) 
    OR (id_from=125298) OR (id_from=17335) OR (id_from=60356)) AND (rating>=6)
    ''',
    con,
)

In [7]:
#выводим id этих пользователей
ratings_request

,id_from,id_to,rating
0,17335,23021,6
1,17335,24497,10
2,17335,31116,10
3,17335,37212,8
4,17335,37885,9
...,...,...,...
91,125299,14884,6
92,125299,31116,6
93,125299,64606,10
94,125299,77036,7


In [8]:

ratings_request.to_sql('rating_request', con, index=False, if_exists='replace')

96

In [9]:
# считываем и выводим датасет info
df_info = pd.read_csv('E:/git_exercise/Final/edges/info.csv')

In [10]:
df_info.head(30)

,id,gender,1,2,3,4,5,6,7,8,9,10,11,12
0,1,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,2,F,1,46,81,6,9,31,22,79,82,75,53,0.675761
2,3,U,4,69,64,2,18,44,54,80,50,89,38,0.927339
3,4,F,3,46,90,9,42,54,59,86,60,65,25,0.176650
4,5,F,3,79,83,1,45,57,1,58,34,23,33,0.553003
5,6,F,1,83,62,4,41,5,41,87,93,60,19,0.776311
6,7,F,2,51,8,13,40,25,53,45,2,70,48,0.942419
7,8,M,5,21,36,17,1,59,15,72,7,81,65,0.721092
8,9,M,4,82,8,3,38,57,63,83,72,89,92,0.246990
9,10,M,5,63,69,6,28,40,1,81,96,40,56,0.654471


In [11]:
#создаем таблицу
df_info.to_sql('info', con, index=False, if_exists='replace')

220970

In [12]:
#объединяем таблицы rating_request и info по условию совпадения id лайкнутых пользователей
data=pd.read_sql(
    '''
    SELECT * FROM info i
    join rating_request rr on rr.id_to=i.id order by rr.id_from
    ''',
    con,
)

In [13]:
data

,id,gender,1,2,3,4,5,6,7,8,9,10,11,12,id_from,id_to,rating
0,23021,F,3,75,62,13,33,5,18,47,51,26,94,0.959493,17335,23021,6
1,24497,F,2,45,93,4,49,18,81,23,30,94,29,0.707485,17335,24497,10
2,31116,M,2,19,53,21,43,45,94,92,36,66,4,0.846278,17335,31116,10
3,37212,F,1,78,26,10,45,1,5,88,3,51,54,0.747732,17335,37212,8
4,37885,F,5,79,68,16,1,58,85,16,38,42,21,0.986321,17335,37885,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,14884,F,4,21,36,7,33,42,15,20,13,2,44,0.039200,125299,14884,6
92,31116,M,2,19,53,21,43,45,94,92,36,66,4,0.846278,125299,31116,6
93,64606,F,1,71,25,22,57,35,50,36,85,64,1,0.741713,125299,64606,10
94,77036,F,1,38,23,18,8,37,96,84,85,17,46,0.396489,125299,77036,7


In [14]:
# столбец id_to дублируется с id, поэтому удаляем его
data2 = data.drop(columns=['id_to'], axis=1)
data2


,id,gender,1,2,3,4,5,6,7,8,9,10,11,12,id_from,rating
0,23021,F,3,75,62,13,33,5,18,47,51,26,94,0.959493,17335,6
1,24497,F,2,45,93,4,49,18,81,23,30,94,29,0.707485,17335,10
2,31116,M,2,19,53,21,43,45,94,92,36,66,4,0.846278,17335,10
3,37212,F,1,78,26,10,45,1,5,88,3,51,54,0.747732,17335,8
4,37885,F,5,79,68,16,1,58,85,16,38,42,21,0.986321,17335,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,14884,F,4,21,36,7,33,42,15,20,13,2,44,0.039200,125299,6
92,31116,M,2,19,53,21,43,45,94,92,36,66,4,0.846278,125299,6
93,64606,F,1,71,25,22,57,35,50,36,85,64,1,0.741713,125299,10
94,77036,F,1,38,23,18,8,37,96,84,85,17,46,0.396489,125299,7


In [ ]:
# Получена раширенная версия rating_request - таблица, где в столбце id_from - 5 заданных пользователей, 
# в столбце id - id лайкнутых пользователей, в столбцах gender и 1...12 - характеристики лайкнутых пользователей  

In [15]:
#Формируем датасет
dataset_1 = pd.DataFrame(data2)
dataset_1

,id,gender,1,2,3,4,5,6,7,8,9,10,11,12,id_from,rating
0,23021,F,3,75,62,13,33,5,18,47,51,26,94,0.959493,17335,6
1,24497,F,2,45,93,4,49,18,81,23,30,94,29,0.707485,17335,10
2,31116,M,2,19,53,21,43,45,94,92,36,66,4,0.846278,17335,10
3,37212,F,1,78,26,10,45,1,5,88,3,51,54,0.747732,17335,8
4,37885,F,5,79,68,16,1,58,85,16,38,42,21,0.986321,17335,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,14884,F,4,21,36,7,33,42,15,20,13,2,44,0.039200,125299,6
92,31116,M,2,19,53,21,43,45,94,92,36,66,4,0.846278,125299,6
93,64606,F,1,71,25,22,57,35,50,36,85,64,1,0.741713,125299,10
94,77036,F,1,38,23,18,8,37,96,84,85,17,46,0.396489,125299,7


In [16]:
#формируем датасет из info, при это убираем из него строки с id тех пользователей, которые уже есть в dataset_1
dataset_2 = pd.DataFrame(df_info)
dataset_2 = dataset_2[~dataset_2['id'].isin(dataset_1['id'])]
dataset_2

,id,gender,1,2,3,4,5,6,7,8,9,10,11,12
0,1,F,1,45,90,15,8,13,63,57,96,42,87,0.934487
1,2,F,1,46,81,6,9,31,22,79,82,75,53,0.675761
2,3,U,4,69,64,2,18,44,54,80,50,89,38,0.927339
3,4,F,3,46,90,9,42,54,59,86,60,65,25,0.176650
4,5,F,3,79,83,1,45,57,1,58,34,23,33,0.553003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220965,220966,U,5,26,65,5,7,18,18,31,65,55,50,0.201921
220966,220967,U,1,85,60,16,30,53,60,53,15,37,58,0.624036
220967,220968,U,1,70,73,22,33,57,88,23,2,22,86,0.867339
220968,220969,U,2,40,45,21,28,27,96,75,77,60,48,0.144322


In [20]:
# Вычисляем косинусное сходство для каждого объекта из dataset_1 с каждым объектом из dataset_2
similarity_matrix = cosine_similarity(dataset_1[['1', '2','3','4','5','6','7','8','9','10','11','12']], dataset_2[['1', '2','3','4','5','6','7','8','9','10','11','12']])

# Находим индексы объектов из dataset_2, которые имеют максимальное сходство с объектами из dataset_1
max_similarities = []
for i in range(len(dataset_1)):
    max_similarity = np.max(similarity_matrix[i])
    max_similarities.append((i, np.where(similarity_matrix[i] == max_similarity)[0][0]))

# Выводим результаты
#for i, j in max_similarities:
#    print(f"User {dataset_1['id'][i]} is most similar to user {dataset_2['id'][j]}")
# Создаем новый датасет с двумя столбцами
results = pd.DataFrame(columns=['id_1', 'id_2','correspondence'])

# Заполняем новый датасет результатами
for i, j in max_similarities:
    results = pd.concat([results, pd.DataFrame([[dataset_1['id'][i], dataset_2['id'][j],similarity_matrix[i][j]]], columns=['id_1', 'id_2', 'correspondence'])], axis=0)
    #results = results.concat({'id_1': dataset_1['id'][i], 'id_2': dataset_2['id'][j], 'correspondence': similarity_matrix[i][j]}, ignore_index=True)
    

print(results)

      id_1    id_2  correspondence
0    23021  164072        0.993348
0    24497   15834        0.996949
0    31116   66745        0.993015
0    37212  144565        0.992188
0    37885  115279        0.992092
..     ...     ...             ...
0    14884  170133        0.983125
0    31116   66745        0.993015
0    64606   43361        0.996561
0    77036    8643        0.994186
0   101503   22127        0.992667

[96 rows x 3 columns]


In [ ]:
#в results получилось 3 столбца: в id_1 - id пользователей, лайкнутых заданными пятью; в id_2 - наиболее схожие с ними
# id пользователей из info, в correspondence - мера сходства

In [21]:
#формируем таблицу 
results.to_sql('results', con, index=False, if_exists='replace')

96

In [37]:
# в финальном запросе добавляем id пяти исходных пользователей, а из results берем столбец id_2 как id предсказанных пользователей
final=pd.read_sql(
    '''
    SELECT rr.id_from, r.id_2 id_predict, r.correspondence FROM results r
    join rating_request rr on rr.id_to=r.id_1 order by rr.id_from,r.correspondence DESC
    ''',
    con,
)

In [38]:
final

,id_from,id_predict,correspondence
0,17335,15834,0.996949
1,17335,2902,0.995318
2,17335,17462,0.995170
3,17335,146612,0.995025
4,17335,136971,0.994612
...,...,...,...
115,125299,66745,0.993015
116,125299,66745,0.993015
117,125299,66745,0.993015
118,125299,22127,0.992667
